#The Project

Nelson CURIER-DUARTE                                           
Franck NGOUNOU KOTCHAP


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from functools import partial
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from skimage import transform
import pdb

##The Model

La classe ConvolutionalNetwork  est composé de 3 champs qui nous seront utiles :</br>
- conv : Initialise les champs de la fonction Conv2d qui est la fonction de convolution.</br>
  * in_channels ici aura une valeur qui sera comprise entre 1 et 3 (1 si noir et blanc et 3 si en couleur (R G B))</br>
  * out_channel</br>
  * kernel size correspond a la taille que la fênetre aura pour chaque case de la matrice de convolution :</br>
par exemple : on a une matrice <table><tr><td>1</td> <td>2</td> <td>3</td></tr><tr><td> 3</td> <td>0</td> <td>1</td></tr> <tr> <td>1</td> <td>1</td> <td>0</td>  </tr><table>et on a un kernel (une matrice) 2*2 composé de chiffre (0 1 | 0 2), donc on va parcourir cette matrice avec un pas (padding que l'on vera après) avec le filtre et en calculant.</br>
par exemple   ici on aura donc la matrice <table><tr> <td>2</td> <td>4</td> </tr> <tr><td>2</td> <td>1</td></tr></table>  car on a 0x1 + 1x2 +3x0 + 0x2 = 2 on suit le même exemple pour le reste</br>
  * padding indique le pas qu'aura le filtre quand il parcourera la matrice cible par exemple si on a (1,1) il la parcoura de 1 en 1  </br>
             
        
- Une fonction d'activation qui nous permettra en quelque sorte de normaliser les valeurs contenu dans la matrice, par exemple la fonction d'activation <i>reLu</i> qui permettra de supprimer les valeur négative de la matrice. </br>
- Une fonction dropout qui est une fonction de normalisation. Cette fonction permet de désactiver certains neurones au hasard et d'éviter donc au réseau de neurone d'apprendre par coeur les images pendant l'entrainement et donc d'avoir de meilleurs résultats.

In [0]:
class ConvolutionalNetwork(nn.Module):
  """ A Convolution Network with dropout and a activation """
  def __init__(self, in_channels, out_channels, kernel_size, padding, dropout_p, activation):
    super(ConvolutionalNetwork, self).__init__()
    self.conv = nn.Conv2d(
        in_channels= in_channels, 
        out_channels= out_channels,
        kernel_size= kernel_size,
        padding= padding
        )
    self.activation = activation
    self.drop = nn.Dropout(
        p = dropout_p
        )

  def forward(self, x):
        x = self.conv(x)
        x = self.drop(x)
        x = F.relu(x)
        x = self.activation(x)
        return x
        


Avec la classe LinearNetwork nous avons la définition d'un réseau de neurone classique.
- layer prend en argument nn.Linear composé du nombre de neurone en entrée et en sortie. En entrée nous avons en général le nombre de pixel contenue dans l'image que nous voulons analyser
par exemple une image 28 * 28 on entre tout simplement 28*28 et en sortie on peut mettre ce que l'on veut.
- Nous avons ensuite une fonction dropout comme vu précédemment ainsi qu'une fonction d'activation.

In [0]:
class LinearNetwork(nn.Module):
  """ A layer network with dropout and an activation function"""
  def __init__(self, in_features, out_features, dropout_p, activation):
    super(LinearNetwork, self).__init__()
    self.layer = nn.Linear(
        in_features = in_features,
         out_features= out_features
         )
    self.drop = nn.Dropout(
        p = dropout_p
    )
    self.activation= activation

  def forward(self, x):
      x = self.layer(x)
      x = self.drop(x)
      x = self.activation(x)
      return x


La class ImageNetwork est un reseau de neurones qui contient:
- 2 ConvolutionalNetwork:
 * La premère (les inputs) prend en entrée un nombre de channels qui continent des images de 7x7 pixels et a pour fonction d'activation une interpolation au plus proche voisin qui double la taille des images.</br> Par exemple pour une matrice 2x2:
  <table>
  <tr> <td>4</td> <td>6</td> </tr> 
  <tr> <td>2</td> <td>1</td> </tr>
  </table>
  l'interpolation au voisin le plus proche dupliquera les valeurs  de la matrice 2x2 pour obtenir une matrice 4x4:
   <table>
   <tr> <td>4</td> <td>4</td> <td>6</td> <td>6</td> </tr>
   <tr> <td>4</td> <td>4</td> <td>6</td> <td>6</td> </tr>  
   <tr> <td>2</td> <td>2</td> <td>1</td> <td>1</td> </tr>
   <tr> <td>2</td> <td>2</td> <td>1</td> <td>1</td> </tr>
   </table>
Il renvoi ensuite un certain nombre de <i>channels</i>.</br>
  * La seconde prend des images de 14x14 et avec la fonction d'activation interpolation au plus proche voisin elle double les images à nouveau. Et renvoi un certain nombre de <i>channels</i>.

- 1 LinearNetwork:
  * Celui-ci prend entre un tenseur de dimension nombre de batch x 28*28 *<i>channels</i> et renvoi une image de 28x28 pixels avec pour fonction d'activation la fonction sigmoïde.

A la fin le model nous revoie une image en 2 dimentions en 28x28.



In [0]:
class ImageNetwork(nn.Module):
   """A image Network which take a 7x7 picture in input and 
     give a 28x28 picture in output
  """
  def __init__(self):
     super(ImageNetwork, self).__init__()
     self.conv_layer=nn.Sequential(
         ConvolutionalNetwork(
             in_channels = 1,
             out_channels = 32,
             kernel_size = 3,
             padding  = 1,
             dropout_p = .6, 
             activation = partial(F.interpolate, size = (14,14)) # the activation double the size of the picture to 14x14
         ),
         ConvolutionalNetwork(
             in_channels = 32,
             out_channels = 64,
             kernel_size = 3,
             padding  = 1,
             dropout_p = .6,
             activation = partial(F.interpolate, size = (28, 28)) # the activation double the size again of the picture to 28x28
         )
     )
     self.linear_layer = nn.Sequential(
         LinearNetwork(
             in_features = 28 * 28 * 64, # the input of the flatten pictures
             out_features = 784, # the flatten picture 28 * 28
             dropout_p = 0,
             activation = torch.sigmoid # Sigmoid(x) = 1 / 1 + exp(-x) 
        )
      )
     

  def forward(self, x): # x = [batch, 1, 7, 7]
   
    x = self.conv_layer(x) # [batch, 64, 28 ,28]
    x = torch.flatten(x, 1) # [batch, 50176]
    x = self.linear_layer(x) # [batch, 784]
    x = x.view(x.shape[0], 1, 28, 28)  #[batch, 1, 28,28]
    return x

##The resize function and draw image function

La fonction resize va nous permettre de redimmensionner de leur taille initial a 7x7. En regardant le code 
   en détail, on remarque que l'on utilise ici le cpu pour optimiser la performance et  après la transformation
   on précide que le tenseur sera stocker sur le processeur cuda 

In [0]:
def resize(x):
  """Resize the 28x28 picture in a 7x7"""
  xx = x.cpu() #[batch, 1, 28,28]
  xx = np.transpose(xx, (2, 3, 1, 0))  #[28,28,1, batch]
  xx = transform.resize(xx.reshape(28, 28, -1), (7, 7)) # resize the picture to 7x7 [7, 7, batch]
  xx = xx.reshape(7, 7, 1, -1) #[7, 7, 1, batch]
  xx = np.transpose(xx, (3, 2, 0, 1) ) # [batch, 1, 7, 7]
  return torch.Tensor(xx).cuda()

def imshow(inp, org,title=None):
    """Imshow for Tensor."""
    inp = torch.cat([inp[0][0].cpu(), org[0][0].cpu()], dim=1) # Concatenate  the two pictures to the horizontal
    plt.imshow(inp.cpu().detach(), cmap='gray')
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

##The training and eval function

La fonction d'évaluation permet de tester le réseau de neurones avec des images qu'il n'a encore jamais vu.
  Danc celle-ci nous affichons a un certains moment l'image durant les 3 étapes crutiale pour bien comprendre le fonctionnement :</br>
L'image avant la transformation au format 7x7 , celle après la transformation et celle après être passé dans le réseau de neurone  

In [0]:
def eval(model, criterion, device, loader, n_batch = -1):
  model.eval()
  loss    = 0
  total_pred = 0
  with torch.no_grad():
      for batch_idx, (data, _) in enumerate(loader):
          if n_batch !=-1 and batch_idx == n_batch:
            break
          data = data.to(device)
          output = model(resize(data))
          if(batch_idx == 0): # show one example
            plt.imshow(resize(data)[0][0].cpu().detach(), cmap = 'gray')
            plt.show()
            imshow(output,data)
          loss        += criterion(output, data).item()  # sum up batch loss
          total_pred  += len(data)
  return loss / total_pred



def train(model, epochs, optimizer, criterion, device, train_loader, test_loader):
  model.train()
  i = 0
  print('0% training')
  eval(model,criterion, device, train_loader,20)
  for epoch in  range(epochs):
    i = 0
    for batch_idx, (data, _) in enumerate(train_loader):
      data = data.to(device)
      optimizer.zero_grad()
      output = model(resize(data))
      loss = criterion(output,data)
      loss.backward()
      optimizer.step()
      if (i % 500 == 0):
        print((epoch/epochs)*100,"% training",sep='')
        print(f'[{epoch:4}] Train eval average loss: {eval(model,criterion, device, train_loader,20):.6}, Test eval average loss: {eval(model,criterion, device,test_loader):.4}')

      i+=1;




##The create loader function

Cette fonction permet tout simplement de télécharger les images dans la base de donnée MNIST de google selon le fait qu'elles soient plus tard utilisé pour l'entrainement ou pour les tests. 

In [0]:
def create_loaders(datasetMNIST):
  """ Created a loader from the Mnist dataset given in argument"""
  mnist_transform = transforms.Compose([
    transforms.ToTensor()
  ])
 
  train_dataset = datasetMNIST(
    root      = '../data',
    train     = True,
    download  = True,
    transform = mnist_transform
  )

  test_dataset = datasetMNIST(
    root      = '../data',
    train     = False,
    download  = True,
    transform = mnist_transform
  )

 
  train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
  test_loader  = DataLoader(test_dataset, batch_size = 64, shuffle = True)

  return train_loader, test_loader

##The Main Function

 C'et ici que nous allons initialiser les paramètre qui seront utilisé par toutes les fonctions que nous avons vu précédemment.
  Tout d'abord nous initialisons les objet train_loaders (pour l'entrainement) et test_loaders (pour les tests) qui contiendront les iamges que nous utiliserons tout au long
  du programme:</br>

  - epochs concerne le nombre d'entrainement que nous allons effectuer.
  - learning rate est tout simplement le taux d'apprentissage , c'est une sorte de bonus que l'on ajoute au calcul de la descente de gradient qui permet de l'accélérer plus ou moins selon sa valeur.
  - device correspond au "support" a utilisé (processeur etc) ici ce sera le processeur cuda.
  - Nous chargons le modèle qui sera utilisé ici et ensuite sur quel device celui-ci sera exécuté.
  - Ensuite nous définissons la fonction que nous voulons utiliser  qui permettra à notre réseau de neurone "d'apprendre. Il en 
  existes plusieurs qui fonctionne plus ou moins bien selon les donnée et autres. Ici on utilisera Adam qui a une excellente performacnce.
  - Nous définissons ensuite la fonction de loss qui permet de calculer la performance du réseau de neurone.
  - Finalement nous pouvons enfin lancer la fonction d'entrainement . 

In [0]:
def main():
  (
   train_loader,
   test_loader
  )=create_loaders(datasets.MNIST)
  epochs = 200
  learning_rate = 1e-2
  
  device = torch.device('cuda')
  model = ImageNetwork()
  model.to(device)
  optimizer = optim.Adam(model.parameters(),  lr = learning_rate)
  criterion = nn.MSELoss()
  train(model, epochs, optimizer, criterion, device, train_loader, test_loader)

In [0]:
main()

##1. Fashion MNIST

C'est le même pricipe que pour le Mnist précedent. On utilise donc la classe ImageNetwork et à la fonction create_loaders on fait appel au dataset FashionMnist.

In [0]:
def fashionMNIST():
  (
      train_loader,
   test_loader
   )=create_loaders(datasets.FashionMNIST)
  epochs = 200
  learning_rate = 1e-2
  
  device = torch.device('cuda')
  model = ImageNetwork()
  model.to(device)
  optimizer = optim.Adam(model.parameters(),  lr = learning_rate)
  criterion = nn.MSELoss()
  train(model, epochs, optimizer, criterion, device, train_loader, test_loader)

In [0]:
fashionMNIST()